In [1]:
run_folder = 'tracks_symlink/desk/routes/H/runs/20'
              

## Imports and Helpers

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython import display
import subprocess
%matplotlib inline

In [3]:
def video_tag(path,width=300,height=240,title=""):
    '''
    generates an HTML fragmentfor displaying the video in path in the notebook
    '''
    return """
    <div style="float:left;padding-left:5px">
    <p>{3}</p>
    <video width="{1}" height="{2}" controls>
      <source src="{0}">
    </video>
    </div>
    """.format(path,width,height,title)

In [4]:
def table_tag(fragments, columns):
    column = 1
    s = '<table><tr>'
    for fragment in fragments:
        s = s + '<td>' + fragment + '</td>'
        column = column + 1
        if column > columns:
            s = s + '</tr><tr>'
            column = 1
    s = s + '</td></tr></table>'
    return s

In [5]:
def big_figure():
    fig = plt.figure()
    fig.set_size_inches(w=15,h=10)
    return fig

## Settings

In [6]:
front_meters_per_odometer_tick = 0.002528
rear_meters_per_odometer_tick = 0.002017
front_wheelbase_width_in_meters = 0.2413
rear_wheelbase_width_in_meters = 0.2667
wheelbase_length_in_meters = 0.33655

## Analysis

In [7]:
route_folder = os.path.dirname(os.path.dirname(run_folder)) # two folders up
route_folder

'tracks_symlink/desk/routes/H'

In [8]:
print(subprocess.check_output('cat '+run_folder+"/config.json",shell=True).decode('utf-8'))


track_name = desk
route_name = H
max_a = 0.5
max_v = 1
steering_k_p = 80
steering_k_i = 0
steering_k_d = 5
v_k_p = 6
v_k_i = 0
v_k_d = 0.5
slip_rate = 0.2
slip_slop = 0.5
t_ahead = 0.3
d_ahead = 0.1
k_smooth = 0.4
prune_max = 0.5
prune_tolerance = 0.02
capture_video = 0
crash_recovery = 0
optimize_velocity = 1



In [9]:
# Create / Show videos


video_tags = []
for f in ['video_left','video_right']:
    mp4_path = os.path.join(run_folder,f+'.mp4')
    if not os.path.exists(mp4_path):
        avi_path = os.path.join(run_folder,f+'.avi')
        if os.path.exists(avi_path):
            s = 'ffmpeg -i "{0}" -vcodec h264 -acodec copy "{1}"'.format(avi_path,mp4_path)
            subprocess.call(s,shell=True)
    if os.path.exists(mp4_path):
        video_tags.append(video_tag(mp4_path,title=f))
    else:
        print("couldn't find ", mp4_path)
display.display(display.HTML(table_tag(video_tags, columns=2)))


couldn't find  tracks_symlink/desk/routes/H/runs/20/video_left.mp4
couldn't find  tracks_symlink/desk/routes/H/runs/20/video_right.mp4


In [25]:
tdt = pd.read_csv(os.path.join(run_folder,'td.csv'),parse_dates=(['timestamp']))


TypeError: 'NoneType' object is not subscriptable

In [18]:
'fl' in tdt.columns

False

In [19]:
def get_td(folder):
    td = pd.read_csv(os.path.join(folder,'td.csv'),parse_dates=(['timestamp']))
    td.t = (td.ms - td.ms.values[0])/1000.
    
        
    #generate some extra columns
    for p in ['fl','fr','bl','br']:
        if not 'odo_'+p in td.columns:
            td['odo_'+p] = td['odo_'+p+'_a'] + td['odo_'+p+'_b']
        td[p] = td['odo_'+p] - td['odo_'+p][0]

    # create td.heading from yaw, represents total turn angle from start
    dy = td.yaw.diff()
    td.heading = np.cumsum(dy + 360*(dy<-190)-360*(dy>190))
    
    # create center wheels
    td.fc = (td.fl+td.fr)/2. # front center
    td.bc = (td.bl+td.br)/2. # back center
    return td


In [20]:
!pwd

/home/brian/car/notebooks


In [21]:
route = get_td(route_folder)
run = get_td(run_folder)
print(run.columns)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
plt.plot(run.heading)

In [ ]:
big_figure()
plt.title('fc wheel distance vs. Steering')
plt.plot(run.fc,run.str,color='g',label='run')
plt.plot(route.fc,route.str,color='b',label='route')
plt.legend();


In [ ]:
i = 1
plt.plot(run.fr[i:],(run.fr.values[i:]-run.fr.values[:-i])/(run.odo_fr_us.values[i:]-run.odo_fr_us.values[:-i]+1),color='g')
#plt.plot(route.fr[i:],route.fr.values[i:]-route.fr.values[:-i],color='b')

In [ ]:
fr = run[['fr','odo_fr_us']]
fr.drop_duplicates(inplace=False)
d=fr.diff()
fig = big_figure()
plt.plot((d.fr/d.odo_fr_us*1E6)[200:230])


In [ ]:
fig = plt.figure()

In [ ]:
fig = plt.figure()
fig.set_size_inches(w=15,h=10)

ratio = 1 #.25
plt.title('wheel clicks vs time')
plt.plot(route.t,route.fl*ratio,color='g',label='fl')
plt.plot(route.t,route.fc*ratio,color='c', label='fc')
plt.plot(route.t,route.fr*ratio,color='r', label='fr')

plt.plot(route.t,route.bl,color='b', label='bl')
plt.plot(route.t,route.bc,color='m', label='bc')
plt.plot(route.t,route.br,color='k', label='br')
plt.legend();

In [ ]:
fig = plt.figure()
fig.set_size_inches(w=15,h=10)

ratio = 1 #.25
plt.title('wheel clicks vs time')
plt.plot(run.t,run.fl*ratio,color='g',label='fl')
plt.plot(run.t,run.fc*ratio,color='c', label='fc')
plt.plot(run.t,run.fr*ratio,color='r', label='fr')

plt.plot(run.t,run.bl,color='b', label='bl')
plt.plot(run.t,run.bc,color='m', label='bc')
plt.plot(run.t,run.br,color='k', label='br')
plt.legend();

In [ ]:
v=np.zeros(len(run),np.float)

In [ ]:
run.columns

In [ ]:
len(run.odo_fl[2:]),len(run.odo_fl_us[1:-1])
#plt.plot(np.diff(run.odo_fl)/np.diff(run.odo_fl_us))
d10 = (run.odo_fl[10:])-(run.odo_fl[0:])
#plt.plot(run.odo_fl,run.odo_fl[10:]-run.odo_fl[0:])
np.max(d10)
#plt.plot(run.odo_fl)

In [ ]:
plt.plot(np.abs(np.diff(run.odo_fr,n=1)))

In [ ]:
le = np.seterr(divide = 'ignore')
with np.errstate(invalid='ignore', divide='ignore'):
    plt.plot(np.divide(np.diff(run.odo_fl,n=1).astype(np.float),np.diff(run.odo_fl_us,n=1).astype(float)))
#np.seterr(le)

In [ ]:
class KalmanFilter:
    def __init__(self, mean, variance):
        self.mean = float(mean)
        self.variance = float(variance)
    def measure(self,mean,variance):
        self.mean = (self.variance*float(mean)+self.mean*variance)/(self.variance+float(variance))
        self.variance = 1/((1/variance)+(1/self.variance))
    def update(self,distance,variance):
        self.mean += distance
        self.variance += variance                                                                    

In [ ]:
def raw_v(odo,us,meters_per_odometer_tick):
    v = np.zeros_like(odo).astype(np.float)
    dx = np.diff(odo).astype(np.float)*meters_per_odometer_tick
    dt = np.diff(run.odo_fl_us).astype(np.float)/1E6
    with np.errstate(invalid='ignore', divide='ignore'):
        v[1:] = np.where(dt > 0, dx/dt,0)
    return v




In [ ]:
run.fr_v = raw_v(run.odo_fr,run.odo_fr_us, front_meters_per_odometer_tick)
run.fl_v = raw_v(run.odo_fl,run.odo_fl_us, front_meters_per_odometer_tick)
run.br_v = raw_v(run.odo_br,run.odo_br_us, rear_meters_per_odometer_tick)
run.bl_v = raw_v(run.odo_bl,run.odo_bl_us, rear_meters_per_odometer_tick)

big_figure()
plt.plot(run.t, run.fr_v)
plt.plot(run.t, run.fl_v)
plt.plot(run.t, run.br_v)
plt.plot(run.t, run.bl_v)


In [ ]:
plt.plot(run.fl,run.fl_v)
    

In [ ]:
plt.plot(np.cumsum(run.ax))
plt.plot(np.cumsum(run.ay))
plt.plot(np.cumsum(run.az))

In [ ]:
def kalman_v(ax,vraw):
    f = KalmanFilter(0,0)
    k_v = []
    for i,v in enumerate(vraw):
        f.update(0.01*ax[i],0.01**2)
        f.measure(v,.02)
        k_v.append(f.mean)
    return np.array(k_v)


In [ ]:
run.fl_kv = kalman_v(run.ax,run.fl_v)
run.fr_kv = kalman_v(run.ax,run.fr_v)
run.bl_kv = kalman_v(run.ax,run.bl_v)
run.br_kv = kalman_v(run.ax,run.br_v)

In [ ]:
run.fc_kv = (run.fl_kv+run.fr_kv)/2
run.fc_v =( run.fl_v+run.fr_v)/2

In [ ]:
v_line = 0.5

big_figure()
plt.subplot(5,1,1)
plt.title('fl')
plt.plot(run.fl_v,color='lightgreen')
plt.plot(run.fl_kv,color='green')
plt.hlines(xmin=0,xmax=len(run.t),y=v_line,linestyles='dashed')

plt.subplot(5,1,2)
plt.title('fr')
plt.plot(run.fr_v,color='lightblue')
plt.plot(run.fr_kv,color='blue')
plt.hlines(xmin=0,xmax=len(run.t),y=v_line,linestyles='dashed')

plt.subplot(5,1,3)
plt.title('bl')
plt.plot(run.bl_v,color='pink')
plt.plot(run.bl_kv,color='red')
plt.hlines(xmin=0,xmax=len(run.t),y=v_line,linestyles='dashed')

plt.subplot(5,1,4)
plt.title('br')
plt.plot(run.br_v,color='gray')
plt.plot(run.br_kv,color='black')
plt.hlines(xmin=0,xmax=len(run.t),y=v_line,linestyles='dashed')

plt.subplot(5,1,5)
plt.title('fc')
plt.plot(run.fc_v,color='orchid')
plt.plot(run.fc_kv,color='indigo')
plt.hlines(xmin=0,xmax=len(run.t),y=v_line,linestyles='dashed')

In [ ]:
big_figure()
#plt.subplot(2,1,1)
plt.plot(run.t ,run.fl_kv, color='green')
plt.plot(run.t, run.fr_kv, color='blue')
plt.plot(run.t, run.fc_kv, color='darkgreen')
#plt.subplot(2,1,2)
plt.plot(run.t, run.bl_kv, color='red')
plt.plot(run.t, run.br_kv, color='black')


In [ ]:
def diff_n(v,n=1):
    rv = np.zeros_like(v,dtype=np.float)
    rv[n:] = np.subtract(v[n:],v[0:-n])
    return rv

In [ ]:
# research velocity
fig = big_figure()
fig.suptitle("esc and acceleration")
plt.subplot(3,1,1)
plt.plot(route.fc, route.esc, color='b',label='route')
plt.plot(run.fc, run.esc, color='g',label='run')
plt.legend()
plt.title("esc vs fc")
plt.subplot(3,1,2)
plt.title('ax vs fc')
plt.plot(run.fc, run.ax,color='g');
plt.plot(route.fc, route.ax,color='b');

big_figure()

plt.title('v vs fl')
plt.plot(run.fc, run.fl_v,color='lightgreen');
plt.plot(run.fc, run.fl_kv,color='g');

big_figure()

plt.title('v vs fr')
plt.plot(run.fc, run.fr_v,color='lightblue');
plt.plot(run.fc, run.fr_kv,color='blue');

In [ ]:
plt.plot(run.t,diff_n(run.heading,2));

In [ ]:
# How does wheel speed difference compare to turn rate?
run.dtheta=diff_n(run.heading,2)
plt.plot(run.fr_kv-run.fl_kv, run.dtheta,"o",alpha=0.2);

In [ ]:
run.dtheta=diff_n(run.heading,2)
plt.plot(run.br_kv-run.bl_kv, run.dtheta,"o",alpha=0.2);

In [ ]:
def bar_chart(d):
    '''
    plots a bar chart for dictionary d
    '''
    plt.bar(range(len(d)), d.values(), align='center')
    plt.xticks(range(len(d)), d.keys(),rotation=30,size=12)
    return plt

In [ ]:
run[1000:1050]

In [ ]:
d = {
    'fl_v':np.std(np.diff(run.fl_v)),
    'fr_v':np.std(np.diff(run.fr_v)),
    'fc_v':np.std(np.diff(run.fc_v)),
    'fl_kv':np.std(np.diff(run.fl_kv)),
    'fr_kv':np.std(np.diff(run.fr_kv)),
    'fc_kv':np.std(np.diff(run.fc_kv)),
    'bl_v':np.std(np.diff(run.bl_v)),
    'br_v':np.std(np.diff(run.br_v)),
    'bl_kv':np.std(np.diff(run.bl_kv)),
    'br_kv':np.std(np.diff(run.br_kv)),
}
bar_chart(d);
plt.title('standard deviations of successive measurements');